%md
### 🔹 Spark DataFrame Creation Explained

- **`spark`**  
  An object of the **SparkSession class**.  
  In Databricks, the `spark` object is created automatically (no need to define it manually with `SparkSession.builder...`).

- **`spark.read`**  
  A **property of the SparkSession object** that returns a **DataFrameReader class object**.  
  This object is responsible for creating DataFrames from external data sources.

- **`format("csv")`**  
  A **method of the DataFrameReader class**.  
  It specifies that the input file format is CSV.

- **`option("header", True)`**  
  A **method of the DataFrameReader class**.  
  It sets how the DataFrame should be created.  
  Here, it tells Spark to treat the first row as the column header.

- **`option("inferSchema", True)`**  
  Another method of the DataFrameReader class.  
  It enables automatic schema inference so Spark detects column data types instead of reading all as strings.

- **`load("path")`**  
  A **method of the DataFrameReader class** that loads the data from the given path.  
  It returns a **DataFrame object**.

- **`limit(1000)`**  
  A **method of the DataFrame class**.  
  It restricts the DataFrame to only the first 1000 rows.

- **`display(fire_df)`**  
  A Databricks function that displays the DataFrame in a tabular format for easy visualization.


In [ ]:
fire_df = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True) 
    .load("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv")
)
display(fire_df)


%md
### 🔹 How to Write SQL Queries on a DataFrame in Databricks

- To run SQL queries on a DataFrame, you first need to **register the DataFrame as a temporary SQL view**.  
- This is done using the `createOrReplaceTempView("view_name")` method.  

---

### 🧩 What `createOrReplaceTempView` Does
- It assigns a **name** to your DataFrame so Spark SQL can treat it like a table.  
- If a view with the same name already exists, it will be **replaced** by the new DataFrame.  
- The view is **temporary**: it only exists during the current Spark session and disappears once the session ends.  

---

### 🔹 Running SQL Queries
- After registering, you can write SQL queries against the view just like you would on a table.  
- In Databricks, you can:
  - Use `%sql` in a notebook cell to run SQL directly.  
  - Or use `spark.sql("...")` inside Python to run the query and get back a DataFrame.  

---

### 🔗 Workflow Summary
1. **DataFrame** → load your dataset into memory.  
2. **Temp View** → register the DataFrame with a name.  
3. **SQL Query** → run SQL statements against that view.  
4. **Result** → Spark returns the query result as a new DataFrame, which can be further transformed or displayed.


In [ ]:
fire_df.createOrReplaceTempView("fire_calls")


In [ ]:
%sql
select * from fire_calls


In [ ]:
%sql
create database if not exists demo_db;


In [ ]:
%sql
show databases


In [ ]:
%sql
create table if not exists 
demo_db.fire_service_calls_tbl(
  Call_Number int,
  Unit_ID string,
  Incident_Number int,
  CallType string,
  Call_Date date,
  Watch_Date date,
  Call_Final_Disposition string,
  Available_DtTm string,
  Address string,
  City string,
  Zipcode_of_Incident int,
  Battalion string,
  Station_Area string,
  Box string,
  OrigPriority string,
  Priority string,
  Final_Priority int,
  ALS_Unit boolean,
  Call_Type_Group string,
  NumAlarms int,
  UnitType string,
  Unit_sequence_in_call_dispatch int,
  Fire_Prevention_District string,
  Supervisor_District string,
  Neighborhood string,
  Location string,
  RowID string,
  Delay double

) using Delta


In [ ]:
%sql
insert into demo_db.fire_service_calls_tbl
select * from fire_calls


In [ ]:
%sql
select * from demo_db.fire_service_calls_tbl
limit(2)


In [ ]:
%sql
select count(*) from demo_db.fire_service_calls_tbl


#### Q1. How many distinct types of calls were made to the fire department ?


In [ ]:
%sql
 select count(distinct calltype) as distinct_call_type_count 
 from demo_db.fire_service_calls_tbl


#### Q2. What were distinct types of calls made to the Fire Department?


In [ ]:
%sql
select distinct calltype as distinct_call_type 
from demo_db.fire_service_calls_tbl ;


#### Q3. Find out all response for delayed times greater than 5 mins ?


In [ ]:
%sql
select * 
from demo_db.fire_service_calls_tbl
where delay > 5


#### Q4. What were the most common call types ?


In [ ]:
%sql
select 
calltype,
count(*) as calltype_count 
from demo_db.fire_service_calls_tbl
group by calltype
order by calltype_count desc


#### Q5. What zip codes accounted for most common calls?


In [ ]:
%sql
select 
zipcode_of_incident,
count(*) as zipcode_count
from demo_db.fire_service_calls_tbl
group by Zipcode_of_Incident
order by zipcode_count desc ;


#### Q6. What San Francisco neighborhoods are in the zip codes 94102 and 94103 ?


In [ ]:
%sql
select zipcode_of_incident , neighborhood 
from demo_db.fire_service_calls_tbl 
where Zipcode_of_Incident == 94102 or Zipcode_of_Incident = 94103


#### Q7. What was the sum of all call alarms, average, min, and max of the call response times ?


In [ ]:
%sql
select sum(numalarms) , avg(delay), min(delay), max(delay) 
from demo_db.fire_service_calls_tbl


#### Q8. How many distinct years of data is in the dataset


In [ ]:
%sql
select distinct extract(year from call_date)as distinct_year 
from demo_db.fire_service_calls_tbl
order by distinct_year


#### Q9. What week of the year in 2018 had the most fire calls ?


In [ ]:
%sql
select weekofyear(to_date(call_date,"MM/dd/yyyy")) week_year ,
count(*) as count 
from demo_db.fire_service_calls_tbl
where extract(year from call_date) = 2018 
group by week_year 
order by count desc


#### Q10. What neighborhoods in San Francisco had the worst response time in 2018?


In [ ]:
%sql
select 
Neighborhood,
delay
from demo_db.fire_service_calls_tbl
where city = 'San Francisco' and extract(year from call_date) = 2018
order by delay desc
